<a href="https://colab.research.google.com/github/otnemrasordep/progGP-ismir2023/blob/main/progmetaltabgenerator_ismir2023tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISMIR 2023 Tutorial: Progressive Metal Guitar Tablature Generation Using Fine-Tuned Transformers



This notebook is part of the ISMIR 2023 tutorial entitled *Transformer-based Symbolic Music Generation: Fundamentals to Advanced Concepts, Stylistic Considerations, Conditioning Mechanisms and Large Language Models*.

It allows users to generate progressive metal riffs, ideas, and snippets of songs by prompting a [Transformer model](https://github.com/YatingMusic/compound-word-transformer) with some initial content.

The output of the notebook comes in the form of Guitar Pro (```.gp5```) files, a symbolic format that represents the song as guitar tablatures (tabs), which encodes notes/playing techniques using information about fret and string numbers. This format also supports bass guitar, drums, piano, and various other instruments. This work is based on a pre-trained model which was trained on a large corpus of over 26,000 songs ([Sarmento et al., 2023](https://dl.acm.org/doi/abs/10.1007/978-3-031-29956-8_17)). This project further fine-tunes the pretrained model using 173 hand-picked progressive metal songs from various different artists and derives from the findings in ([Loth et al., 2023](https://arxiv.org/abs/2307.05328)).

The contents within this notebook are part of a on-going collaboration, and account for contributions from [Pedro Sarmento](https://github.com/otnemrasordep), [Adarsh Kumar](https://github.com/AdarshKumar712), [Jack Loth](https://github.com/JackJamesLoth) and [Dadabots](https://github.com/dada-bots).

Feel free to use this notebook for research and creative purposes, but please take into consideration that the authors do not have the rights for the songs that were used to train the models. **We advise against the use of this notebook for commercial purposes and take no responsability.**

## 1. Setting Up, Requisites and Dependencies

**1.1. In order to actually view and listen to the generated tabs, you will need software that is capable of reading ```.gp5``` files. [Guitar Pro](https://www.guitar-pro.com/) is a paid software, but free alternatives such as [MuseScore](https://musescore.org/en) and [TuxGuitar](https://sourceforge.net/projects/tuxguitar/) are able to read the files as well. We invite you to download either one of these software in order to follow the tutorial and interact with the model.**

1.2 Clone the github repository into your Google Drive. Approximate size of the repository is 1.7GB.

In [ ]:
# install git lfs and clone the repo into your drive
!git lfs install
!git clone https://github.com/otnemrasordep/progGP-ismir2023

1.3 Change directory into appropriate path where repository was cloned and download model checkpoints.

In [ ]:
%cd progGP-ismir2023

1.4 Connect to a GPU by going to "*Runtime > Change Runtime Type > Hardware accelerator: T4 GPU*". This notebook is built to work with GPU, but with some minor alterations it will work on a CPU-only machine (taking much longer to generate the riffs, of course).

1.5 Install Python dependencies. You will need these three packages to be able to use the model.

**Please note that these packages might take a while to download and install. You might also need to restart your runtime after package installations.**



In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install PyGuitarPro==0.6

In [ ]:
!pip3 install torch torchvision

1.6 Checking if that GPU power is finally available and everything is set correctly.

In [ ]:
import torch
print(f'Do I have my GPU now? \n  {torch.cuda.is_available()}')
print(f'How many GPUs do I possess now? \n  {torch.cuda.device_count()}')

## 2. The DadaGP dataset: a TABifesto



The [DadaGP dataset](https://archives.ismir.net/ismir2021/paper/000076.pdf) was introduced at ISMIR 2021.

It contains over than 26,000 songs in tablature/Guitar Pro format (```.gp5```) and a dedicated token format (```.txt```). To convert between the two formats, we resort on our dedicated [encoder/decoder tool](https://github.com/dada-bots/dadaGP).

With this release, we intended to foster tablature research within the MIR community, for we believe this format is much better suited for guitar players, while encapsulating similar levels of information as other symbolic music formats (e.g. MIDI).

Here's a little illustrative example of how the encoder/decoder tool goes from tablature to token format.

<div>
<img src="https://drive.google.com/uc?export=view&id=1NXW6c8sR0-8Av1vriYoX8cqGYM5C7YII" width="500"/>
</div>

## 3. The Pre-Trained Model

We used a Transformer-XL based model as a backbone architecture, as presented in [Pop Music Transformer](https://dl.acm.org/doi/10.1145/3394171.3413671).

The architecture is available at this [repository](https://github.com/YatingMusic/compound-word-transformer).

We trained this model on the whole DadaGP dataset for a total of 200 epochs.

## 4. The Fine-Tuned Model

The progressive metal artists used to fine-tune the model include: Born Of Osiris, Between The Buried And Me, The Contortionist, Cynic, Destrage, Dillinger Escape Plan, Gojira, Leprous, Mastodon, Necrophagist, Ne Obliviscaris, Opeth, Periphery, Protest The Hero, Sikth, Thank You Scientist, The Human Abstract and The Ocean.

For a complete list and access to this subset, please refer to [ProgGP](https://github.com/otnemrasordep/ProgGP/tree/main).

Here is a word-cloud representation of the artists in the subset:


<div>
<img src="https://drive.google.com/uc?export=view&id=19tjyshd0Ei2GTOhlQLEvNuXx8AJoL44x" width="1000"/>
</div>

We fine-tuned the pre-trained model on this subset for 15 epochs.

For more details about the process, please refer to our [paper](https://arxiv.org/abs/2307.05328).

## 5. Generating Riffs 🤘

### Step 0: import the helper functions
The helper functions will take care of:
1. read the prompt in ```.gp5``` format (stored in ```progGP-ismir2023/tab_prompts```)
2. convert the prompt into token format (```.txt```) to be used by the model (stored in ```progGP-ismir2023/tab_prompts_encoded)```
3. generate the riffs inspired by the prompt in token format (```.txt``` stored in ```progGP-ismir2023/genetared_tabs_encoded```)
4. convert the generated riffs into ```.gp5``` for download (stored at ```progGP-ismir2023/generated_tabs```)

In [8]:
import helpergp as hgp

### Step 1: select a riff or note(s) to prompt the model with
The model works best by taking an initial idea and generating a likely continuation to it. For this notebook, a few different riffs from various songs have been prepared for you to prompt the model with. You may also write your own riff in Guitar Pro, MuseScore, or any other similar software and use it as well! Just ensure that it is in ```.gp5``` format and put it in the ```progGP-ismir2023/tab_prompts``` directory.

The following code will encode the ```.gp5``` file to a ```.txt``` file that is readable by the model (steps 0.1 and 0.2).

In [ ]:
# Choose a prompt from the following options:

# 'marigold' (Marigold - Periphery)
# 'animalbones' (Animal Bones - Protest The Hero)
# 'sequoiathrone' (Sequoia Throne - Protest The Hero)
# 'mines' (Mines - Serval)
# 'theprice' (The Price - Leprous)
# 'sunofnothing' (Sun of Nothing - Between The Buried and Me)
# 'antsofthesky' (Ants of the Sky - Between The Buried and Me)
# 'cadger' (Cadger - October Horse)
# 'andromeda' (Andromeda - October Horse)
# 'bloodanthunder' (Blood and Thunder - Mastodon)
# 'stabwound' (Stabwound - Necrophagist)

# For a custom prompt, just save your GP5 file as '{name}_prompt.gp5' in the tab_prompts dir and put the name here!

# Select the prompt
tab_prompt = 'marigold'

# Encode the .gp5 file to .txt
hgp.encodeTabPrompt(tab_prompt)

### Step 2: configure the model's output
Determine how many samples you would like to generate (```num_samples```) and how long those samples should be, in terms of the number of generated tokens (```num_tokens```). For the latter we recommend ```512```, ```1024``` or ```2048``` tokens, but feel free to play around and experiment!

In [ ]:
# Select how many samples to generate
num_samples = 3
# Select the number of tokens per generation
num_tokens = 512

### Step 3: generate some riffs
This will generate the samples as ```.txt``` files, then decode them to ```.gp5``` files (steps 1.3 and 1.4). Once this is done, you can download the ```.gp5``` files from ```progGP-ismir2023/generated_tabs```) and listen to them  in Guitar Pro or TuxGuitar or Musescore! 🤘

**Please note that the model will overwrite older generations with new ones, so a good practice is to download them after being created.**

In [ ]:
# Let's get those riffs flowing lml
hgp.generateSamples(num_samples, tab_prompt, num_tokens)

## References
1. Sarmento, P., Kumar, A., Carr, C., Zukowski, Z., Barthet, M.,Yang, Y.H. (2021). DadaGP: A Dataset of Tokenized GuitarPro Songs for Sequence Models. In: Proc. of the 22nd International Society for Music Information Retrieval (ISMIR) Conference 2021 (pp. 610–618)

2. Sarmento, P., Kumar, A., Chen, Y. H., Carr, C. J., Zukowski, Z., & Barthet, M. (2023). GTR-CTRL: Instrument and Genre Conditioning for Guitar-Focused Music Generation with Transformers. In: Artificial Intelligence in Music, Sound, Art and Design: 12th International Conference, EvoMUSART 2023, Held as Part of EvoStar 2023 (pp. 260-275). Springer Nature Switzerland.

3. Loth, J., Sarmento, P., Carr, C. J., Zukowski, Z., & Barthet, M. (2023). ProgGP: From GuitarPro Tablature Neural Generation To Progressive Metal Production. In: Proc. of the 16th International Symposium on Computer Music Multidisciplinary Research (CMMR).

4. Huang, Y.-S. and Yang, Y.-H. (2020). Pop Music Transformer: Beat-based Modeling and Generation of Expressive Pop Piano Compositions. In: Proc. of the 28th ACM Int. Conf. Multimedia (pp. 1180-1188).



